In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.basics import *

## Feature engineering

In [3]:
PATH = Path('data/rossmann')

In [4]:
PATH

PosixPath('data/rossmann')

In [5]:
PATH.ls()

[PosixPath('data/rossmann/googletrend.csv'),
 PosixPath('data/rossmann/weather.csv'),
 PosixPath('data/rossmann/rossmann.tgz'),
 PosixPath('data/rossmann/test.csv'),
 PosixPath('data/rossmann/joined'),
 PosixPath('data/rossmann/train_clean'),
 PosixPath('data/rossmann/train.csv'),
 PosixPath('data/rossmann/state_names.csv'),
 PosixPath('data/rossmann/sample_submission.csv'),
 PosixPath('data/rossmann/test_clean'),
 PosixPath('data/rossmann/store_states.csv'),
 PosixPath('data/rossmann/joined_test'),
 PosixPath('data/rossmann/df'),
 PosixPath('data/rossmann/store.csv')]

In [6]:
# pd.read_csv
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(PATH/f'{fname}.csv', low_memory=False) for fname in table_names]

train, store, store_states, state_names, googletrend, weather, test = tables

len(train), len(test)

(1017209, 41088)

In [7]:
train.describe(include = 'all') 

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1017209,1.017209e+06
unique,NaN,NaN,942,NaN,NaN,NaN,NaN,4,NaN
top,NaN,NaN,2014-01-01,NaN,NaN,NaN,NaN,0,NaN
freq,NaN,NaN,1115,NaN,NaN,NaN,NaN,986159,NaN
mean,5.584297e+02,3.998341e+00,NaN,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,NaN,1.786467e-01
std,3.219087e+02,1.997391e+00,NaN,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,NaN,3.830564e-01
min,1.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00
25%,2.800000e+02,2.000000e+00,NaN,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
50%,5.580000e+02,4.000000e+00,NaN,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
75%,8.380000e+02,6.000000e+00,NaN,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,NaN,0.000000e+00


In [8]:
train.dtypes

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object

In [9]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [10]:
train.tail()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1
1017208,1115,2,2013-01-01,0,0,0,0,a,1


In [11]:
# Change data type to boolean
train.StateHoliday = train.StateHoliday != '0'
test.StateHoliday = test.StateHoliday != '0'

In [12]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,1
1,2,5,2015-07-31,6064,625,1,1,False,1
2,3,5,2015-07-31,8314,821,1,1,False,1
3,4,5,2015-07-31,13995,1498,1,1,False,1
4,5,5,2015-07-31,4822,559,1,1,False,1


In [13]:
train.tail()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1017204,1111,2,2013-01-01,0,0,0,0,True,1
1017205,1112,2,2013-01-01,0,0,0,0,True,1
1017206,1113,2,2013-01-01,0,0,0,0,True,1
1017207,1114,2,2013-01-01,0,0,0,0,True,1
1017208,1115,2,2013-01-01,0,0,0,0,True,1


In [14]:
# df.merge
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                     suffixes=("", suffix))

In [15]:
weather.describe(include = 'all') 

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
count,15840,15840,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,...,15459.000000,15459.000000,15459.000000,15840.000000,15840.000000,3604.000000,15840.000000,14667.000000,11889,15840.000000
unique,16,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,NaN
top,Hamburg,2013-10-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rain,NaN
freq,990,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6167,NaN
mean,NaN,NaN,14.644129,10.388952,6.198990,8.587816,6.205808,3.626136,93.659596,74.282891,...,24.057572,12.239796,7.025163,22.766604,11.972222,48.864317,0.831718,5.551306,NaN,175.896717
std,NaN,NaN,8.646012,7.379260,6.526391,6.244780,6.086768,6.128390,7.678530,13.486552,...,8.976799,5.067944,4.980602,8.988618,5.872844,13.026954,2.513506,1.687710,NaN,101.588872
min,NaN,NaN,-11.000000,-13.000000,-15.000000,-14.000000,-15.000000,-73.000000,44.000000,30.000000,...,0.000000,0.000000,0.000000,3.000000,2.000000,21.000000,0.000000,0.000000,NaN,-1.000000
25%,NaN,NaN,8.000000,4.000000,1.000000,4.000000,2.000000,-1.000000,90.750000,65.000000,...,14.000000,10.000000,3.000000,16.000000,8.000000,39.000000,0.000000,5.000000,NaN,80.000000
50%,NaN,NaN,15.000000,11.000000,7.000000,9.000000,7.000000,4.000000,94.000000,76.000000,...,31.000000,11.000000,7.000000,21.000000,11.000000,48.000000,0.000000,6.000000,NaN,202.000000
75%,NaN,NaN,21.000000,16.000000,11.000000,13.000000,11.000000,8.000000,100.000000,85.000000,...,31.000000,14.000000,10.000000,27.000000,14.000000,55.000000,0.250000,7.000000,NaN,256.000000


In [16]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [ ]:
state_names.describe(include = 'all') 

In [ ]:
state_names.head()

In [ ]:
weather = join_df(weather, state_names, "file", "StateName")

In [ ]:
weather.describe(include = 'all') 

In [ ]:
weather.head()

In [ ]:
googletrend.describe()

In [ ]:
googletrend.describe(include = 'all') 

In [ ]:
googletrend.head()

In [ ]:
# create new column from split text
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = "HB,NI"

In [ ]:
googletrend.head()

In [ ]:
# datepart to add dummy column
def add_datepart(df, fldname, drop=True, time=False):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64
        
    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 
           'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
weather.head()

In [ ]:
googletrend.head()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# add datepart to every columns
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

In [ ]:
weather.head()

In [ ]:
googletrend.head()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# handle special case
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [ ]:
# instead of inner join, do outer join and then check for null
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

In [ ]:
store.head()

In [ ]:
train.head()

In [ ]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]), len(joined_test[joined_test.StoreType.isnull()])

In [ ]:
joined = join_df(joined, googletrend, ["State", "Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State", "Year", "Week"])
len(joined[joined.trend.isnull()]), len(joined_test[joined_test.trend.isnull()])

In [ ]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]), len(joined_test[joined_test.trend_DE.isnull()])


In [ ]:
joined = join_df(joined, weather, ["State", "Date"])
joined_test = join_df(joined_test, weather, ["State", "Date"])
len(joined[joined.Mean_TemperatureC.isnull()]), len(joined_test[joined_test.Mean_TemperatureC.isnull()])

In [ ]:
# drop duplicate column
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

In [ ]:
# fillna
for df in (joined, joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)
    
    

In [ ]:
# add column event to date
for df in (joined, joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                    month=df.CompetitionOpenSinceMonth, 
                                                    day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [ ]:
# clear out of lower range data by set to 0
for df in (joined, joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [ ]:
# clear out of higher range data by set to limit (24)
for df in (joined, joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

In [ ]:
# If needed, uncomment:
# ! pip install isoweek

In [ ]:
from isoweek import Week

In [ ]:
# add column event to date
for df in (joined, joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [ ]:
# save to disk
joined.to_pickle(PATH/'joined')
joined_test.to_pickle(PATH/'joined_test')

## Durations

In [ ]:
# add column by relationship between row
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []
    
    for s, v, d in zip(df.Store.values, df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [ ]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [ ]:
#df = train[columns]
df = train[columns].append(test[columns])

In [ ]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df = df.set_index("Date")

In [ ]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [ ]:
# Fillna
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

In [ ]:
df.head()

In [ ]:
# rolling quantity
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [ ]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

In [ ]:
bwd.drop('Store', 1, inplace=True)
bwd.reset_index(inplace=True)

In [ ]:
fwd.drop('Store', 1, inplace=True)
fwd.reset_index(inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [ ]:
df.drop(columns, 1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_pickle(PATH/'df')

In [ ]:
df.dtypes

In [ ]:
df["Date"] = pd.to_datetime(df.Date)

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
joined = pd.read_pickle(PATH/'joined')
joined_test = pd.read_pickle(PATH/'joined_test')

In [ ]:
joined = join_df(joined, df, ['Store', 'Date'])
joined_test = join_df(joined_test, df, ['Store', 'Date'])


In [ ]:
joined = joined[joined.Sales != 0]

In [ ]:
# Backup
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [ ]:
joined.to_pickle(PATH/'train_clean')
joined_test.to_pickle(PATH/'test_clean')